In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
vocab_size = 62549 + 1 # Atleast 3 count needed for word

In [4]:
!rsync -P "/content/gdrive/My Drive/NLP/gdrive.py" .

gdrive.py
            964 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=0/1)


In [0]:
from gdrive import download_file_from_google_drive
import pickle

In [0]:
download_file_from_google_drive('178Ulc0pKZCFVMKK0P4V3ldZaxpp35Lh0', '/content/tokenizer.pickle')
# download_file_from_google_drive('1uRkPp8cFgE5pD3I6Ge456pO72tMrmHWa', '/content/model.hdf5')

In [0]:
from glob import glob

In [8]:
glob('/content/gdrive/My Drive/NLP/Word2Vec/training_logs/Word2Vec-Nepali/checkpoints/*')[-1]

'/content/gdrive/My Drive/NLP/Word2Vec/training_logs/Word2Vec-Nepali/checkpoints/model037.hdf5'

In [0]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [0]:
# tokenizer.word_counts

In [0]:
# sorted(tokenizer.word_counts.items(), key= lambda x: x[1], reverse=True)

In [0]:
from tensorflow.python.keras.models import load_model

In [0]:
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras.layers import Input, Dense, Reshape
from tensorflow.python.keras.layers import Embedding
from tensorflow.python.keras.layers.merge import dot
from tensorflow.python.keras.models import Model,load_model
from tensorflow.python.keras.initializers import TruncatedNormal
import numpy as np

In [0]:
class Word2Vec(Layer):
  
  def __init__(self, embed_dim, vocab_size, **kwargs):
    self.embed_dim = embed_dim
    self.vocab_size = vocab_size
    super(Word2Vec, self).__init__(**kwargs)
  
  def build(self, input_shape):
    stddev = 1/np.sqrt(self.embed_dim)
    initializer = TruncatedNormal(stddev=stddev)
    # word embedding shape (10000,300) for my use
    # word embedding input=> (batch_size,sequence_length) output => (batch_size, sequence_length, 300(output_dim))
    self.word_embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embed_dim, 
                     input_length=1, name="word_embedding", embeddings_initializer=initializer)
    self.context_embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embed_dim, 
                     input_length=1, name="context_embedding", embeddings_initializer=initializer)
#     self.trainable_weights=[self.word_embedding, self.context_embedding]
    super(Word2Vec, self).build(input_shape)
  
  def call(self, inputs, **kwargs):
    target_input, context_input = inputs
    target = self.word_embedding(target_input)
    context = self.context_embedding(context_input)
    # shape target: (batch_size, sequence_length, embed_size) => (batch_size,1,300)
    # shape context: (batch_size, sequence_length, embed_size) => (batch_size,1,300)
    dot_product = dot([target,context], axes=(2,2))
    dot_product = Reshape(target_shape=(1,))(dot_product)
    output = Dense(1, activation='sigmoid')(dot_product)
    return output
  
  def get_config(self):
    config = super(Word2Vec, self).get_config()
    config.update({
        'embed_dim': self.embed_dim,
        'vocab_size': self.vocab_size
    })
    return config
  
custom_layers = {
    Word2Vec.__name__: Word2Vec
}

In [19]:
model = load_model(glob('/content/gdrive/My Drive/NLP/Word2Vec/training_logs/Word2Vec-Nepali/checkpoints/*')[-1], custom_objects=custom_layers)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# model.load_weights('/content/gdrive/My Drive/NLP/Word2Vec/training_logs/Word2Vec-Nepali/checkpoints/model037.hdf5')

In [21]:
model.summary()

Model: "Word2Vec"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
target_input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
context_input (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
word2_vec (Word2Vec)            (None, 1)            37530000    target_input[0][0]               
                                                                 context_input[0][0]              
Total params: 37,530,000
Trainable params: 37,530,000
Non-trainable params: 0
__________________________________________________________________________________________________


In [0]:
word2vec = model.layers[2]

In [0]:
word_embedding = word2vec.word_embedding.get_weights()[0][1:]

In [24]:
word_embedding.shape

(62549, 300)

In [0]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [0]:
# cosine distance = 1 - cosine similarity
distance_matrix = cosine_distances(word_embedding)

In [27]:
distance_matrix.shape

(62549, 62549)

In [0]:
# sorted(tokenizer.word_counts.items(), key= lambda x: x[1], reverse=True)

In [29]:
tokenizer.word_index['छ']

1

In [0]:
distance_matrix[tokenizer.word_index['नेपाल']-1].argsort()
import random

In [0]:
vocab_words = np.array(sorted(tokenizer.word_counts.items(), key= lambda x: x[1], reverse=True))[0:vocab_size-1,0]

In [0]:
# word = random.choice(vocab_words)

In [0]:
def random_similar():
  word = random.choice(vocab_words)
  similar_words = {word: [tokenizer.index_word[id] for id in distance_matrix[tokenizer.word_index[word]-1].argsort()[1:6]+1]}
  print(similar_words)

In [0]:
def word_similar(word, count=6):
  similar_words = {word: [tokenizer.index_word[id] for id in distance_matrix[tokenizer.word_index[word]-1].argsort()[1:count]+1]}
  print(similar_words)

In [170]:
word = 'बस'
word_similar(word)

{'बस': ['मिनिबस', 'ट्रक', 'मिनिट्रक', 'जिपका', 'टेम्पो']}


In [172]:
word_similar('जनकपुर', 10)

{'जनकपुर': ['दैलेख–', 'झोराहाट', 'अञ्चल', 'जेसिजका', 'जगदीशप्रसाद', 'रेसुओ', 'जनकपुरको', 'वनबासा', 'डोटीको']}


In [0]:
# tokenizer.word_index['काठमाडौँ']

In [0]:
# query = word_embedding[tokenizer.word_index['राजा']-1].reshape(1, -1) - word_embedding[tokenizer.word_index['रानी']-1].reshape(1, -1) + word_embedding[tokenizer.word_index['महिला']-1].reshape(1, -1)

In [0]:
# vocab = np.array(sorted(tokenizer.word_counts.items(), key= lambda x: x[1], reverse=True))[0:62549,0]

In [0]:
# scores = {}
# for i, word in enumerate(vocab_words):
#   scores[word] = cosine_distances(query, word_embedding[tokenizer.word_index[word]-1].reshape(1, -1))

In [0]:
# cosine_similarity(query, word_embedding[tokenizer.word_index['पुरुष']-1].reshape(1, -1))

In [0]:
# sorted(scores.items(), key=lambda x: x[1], reverse=False)[:20]